In [12]:
import pickle
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.models import save_model
from keras.models import load_model
import os

X = pickle.load(open("../data/X_data_random.pkl","rb"))
y = pickle.load(open("../data/y_data_random.pkl","rb"))
word2id, id2word = pickle.load(open("../data/word2idandid2word.pkl","rb"))
label2id, id2label = pickle.load(open("../data/label2idandid2label.pkl","rb"))
word2vec_matrix = pickle.load(open("../data/word2vec_matrix.pkl","rb"))


index1 = 50000
index2 = 70000
X_train, X_val, X_test = X[:index1], X[index1:index2], X[index2:]
y_train, y_val, y_test = y[:index1], y[index1:index2], y[index2:]

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

padding_len = 200

with tf.device('/gpu:0'):
    model_input = Input(shape=(padding_len,))
    
    wordembed = Embedding(len(word2vec_matrix), 300, weights=[word2vec_matrix], input_length=padding_len, trainable=False)(model_input)
    wordembed = Flatten()(wordembed)
    sen2vec = Dense(300, activation='tanh')(wordembed)
    sen2vec = Dropout(0.5)(sen2vec)
    model_output = Dense(len(label2id), activation='softmax')(sen2vec)
    model = Model(inputs=model_input, outputs=model_output)
    model.summary()
    checkpoint = ModelCheckpoint("../PCmodel/ANN.h5", monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    model.fit(X_train, y_train, batch_size=200, epochs=5, validation_data=(X_val, y_val), callbacks=[checkpoint])
    model.save("../PCmodel/ANN.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 200, 300)          68857800  
_________________________________________________________________
flatten_6 (Flatten)          (None, 60000)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 300)               18000300  
_________________________________________________________________
dropout_5 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 2408      
Total params: 86,860,508
Trainable params: 18,002,708
Non-trainable params: 68,857,800
_______________________________________________________

In [13]:
y_pred = model.predict(X_test)
y1 = [np.argmax(i) for i in y_test]
y2 = [np.argmax(i) for i in y_pred]

from sklearn import metrics

print('precision:')
print(metrics.precision_score(y1,y2,average='micro'))
print(metrics.precision_score(y1,y2,average='macro'))

print('recall:')
print(metrics.recall_score(y1,y2,average='micro'))
print(metrics.recall_score(y1,y2,average='macro'))

print('f1:')
print(metrics.f1_score(y1,y2,average='micro'))
print(metrics.f1_score(y1,y2,average='macro'))

precision:
0.7177
0.7167971154514964
recall:
0.7177
0.7166342378520689
f1:
0.7177
0.7162168305007888
